## NASA Center Analysis

#### TODO: 
- improve quality checks because not that reliable


<details>
<summary>VARIABLES</summary>

| Variable Name       | Long Name                                          | Variable Category | Units     | Description                                                                                                                                                          |
| ------------------- | -------------------------------------------------- | ----------------- | --------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| tmax_days_35C       | \# days Tmax ≥35°C                                 | extreme index     | \# days   | Number of days, per year, with Tmax >=35C                                                                                                                            |
| tmax_days_90th      | \# days Tmax ≥90th Percentile                      | extreme index     | \# days   | Number of days, per year, with Tmax 90th percentile. 90th percentile calculated using all daily tmax values from 1995-2014.                                          |
| tmax_days_95th      | \# days Tmax ≥95th Percentile                      | extreme index     | \# days   | Number of days, per year, with Tmax 95th percentile. 95th percentile calculated using all daily tmax values from 1995-2014.                                          |
| tmax_days_99th      | \# days Tmax ≥99th Percentile                      | extreme index     | \# days   | Number of days, per year, with Tmax 99th percentile. 99th percentile calculated using all daily tmax values from 1995-2014.                                          |
| Hottest_Tmax        | Hottest Tmax of the Year (°C)                      | extreme index     | degrees C | Hottest Tmax value every year                                                                                                                                        |
| Max_DTR             | Largest Diurnal Temperature Range of the Year (°C) | extreme index     | degrees C | largest diurnal temperature range (tmax minus tmin) each year                                                                                                        |
| tmin_tropnights_20C | \# days Tmin ≥20°C                                 | extreme index     | \# days   | Number of days, per year with Tmin >=20C                                                                                                                             |
| tmin_frostdays_0C   | \# days Tmin ≤0°C                                  | extreme index     | \# days   | Number of days per year with Tmin <=0C                                                                                                                               |
| Coldest_Tmin        | Coldest Tmin of the Year (°C)                      | extreme index     | degrees C | Coldest minimum temperature each year                                                                                                                                |
| prec_days_dry       | \# days with precipitation ≤0.001 in               | extreme index     | \# days   | Number of days, per year, where precipitation <=1e-3 inches                                                                                                          |
| prec_days_oneinch   | \# days with precipitation ≥1 in                   | extreme index     | \# days   | Number of days, per year, where precipitation >=1 inch                                                                                                               |
| prec_days_90th      | \# days with precipitation ≥90th Percentile        | extreme index     | \# days   | Number of days, per year, where precipitation >=90th percentile. 90th percentile calculated usingd all daily precipitation values (dry days EXCLUDED) from 1995-2014 |
| prec_days_95th      | \# days with precipitation ≥95th Percentile        | extreme index     | \# days   | Number of days, per year, where precipitation >=95th percentile. 95th percentile calculated usingd all daily precipitation values (dry days EXCLUDED) from 1995-2014 |
| prec_days_99th      | \# days with precipitation ≥99th Percentile        | extreme index     | \# days   | Number of days, per year, where precipitation >=99th percentile. 99th percentile calculated usingd all daily precipitation values (dry days EXCLUDED) from 1995-2014 |
| tmax_annave         | Annual Average Tmax (°C)                           | annual average    | degrees C | Annual average maximum daily temperature                                                                                                                             |
| tmin_annave         | Annual Average Tmin (°C)                           | annual average    | degrees C | Annual average minimum daily temperature                                                                                                                             |
| prec_annave         | Annual Total Precipitation (mm)                    | annual SUM        | degrees C | Annual SUM of precipitation                                                                                                                                          |</details>

In [1]:
# Imports
import os
import warnings
import zipfile
import numpy as np
import pandas as pd
import pandasql as psql

# Suppress warnings
warnings.filterwarnings('ignore')

## Initialization

In [2]:
path = 'data/updated_extremes.zip'  # data directory
center = 'LARC'.upper()    # NASA center to analyze
only_future = True         # flag to use only 2020-2099
ssp = ['ssp126', 'ssp245', 'ssp370'] # scenarios to use

In [3]:
# DO NOT CHANGE THIS CELL
# File name convention: variable.ssp###.CENTER.csv

# NASA Centers
centers = sorted(['AMES', 'GSFC', 'JPL', 'KSC', 'MSFC', 'MAF', 'GISS',
                  'LARC', 'SSC', 'GRC', 'WFF', 'JSC', 'WSTF', 'AFRC'])

# Check if the provided center is valid
if center not in centers:
    raise ValueError(f'{center} not in {centers}')

# Variable unit: number of DAYS when... assume others are celsius
day_unit = ['days', 'tropnights', 'FWI']

# Time periods: 10 years before+after a decade
time_periods = {'short': (2020, 2049),  # 2030's: 2020-2029, 2030-2039, 2040-2049
                'mid':   (2040, 2069),  # 2050's: 2040-2049, 2050-2059, 2060-2069
                'long':  (2070, 2099),  # 2080's: 2070-2079, 2080-2089, 2090-2099
                }

old_filename_conv = ['FWI'] # variable_center_ssp.csv 
# filename.split('_')

# Get Files/Data

In [4]:
def get_files(path: str, center: str):
    '''Returns list of all csv files in the directory that contain the center and ssp name'''
    # return [os.path.join(path, f) for f in os.listdir(path) 
    #          if center in f and any(s in f for s in ssp) and f.endswith('.csv')]
    with zipfile.ZipFile(path, 'r') as zip_ref:
        files = [f.filename for f in zip_ref.filelist 
                 if center in f.filename and any(s in f.filename for s in ssp) and
                 f.filename.endswith('.csv') and not f.filename.startswith('__MACOSX/')]
        return files

def check_df_consistency(df_list: list[pd.DataFrame]):
    '''Returns T/F if all dataframes in the list have the same column names and index values'''
    if not df_list:
        return False
    
    # Get reference column names and index values from the first dataframe
    ref_cols, ref_index = list(df_list[0].columns), list(df_list[0].index)
    
    # Check if all other dataframes have the same column names and index values
    return all(list(df.columns) == ref_cols and list(df.index) == ref_index 
               for df in df_list[1:])

def label_term(year: int):
    '''Returns list of time period labels for a given year'''
    return [t for t, (s, e) in time_periods.items() if s <= year <= e]


def preprocess(filename: str, only_future: bool=True):
    '''Returns a preprocessed pandas DataFrame from a csv file'''
    with zipfile.ZipFile(path, 'r') as zip_ref:
        with zip_ref.open(filename) as file:
            df = pd.read_csv(file)
            df.columns = df.columns.str.lower()
    
    name = filename.split('/')[-1][:-4]
    if any(v in name for v in old_filename_conv): # variable_center_ssp.csv 
        name = name.split('_')
        var_, ssp_ = '_'.join(name[:-2]), name[-1]
    else: # variable.ssp###.CENTER.csv
        name = name.split('.')
        var_, ssp_ = name[0], name[1]

    # Add new columns: term, scenario, and variable
    df.insert(0, 'term', df.years.apply(label_term))
    df.insert(0, 'scenario', ssp_)
    df.insert(0, 'variable', var_ + ('_days' if any(d in filename for d in day_unit) else '_real'))
    
    # Explode the 'term' column (in case a year belongs to multiple terms)
    df = df.explode('term')

    # Remove rows with NaN terms if only_future is True, otherwise return all rows
    return df.dropna(subset=['term']) if only_future else df # nan's (unlabeled) assumed to be past data


def calculate_statistics(df: pd.DataFrame):
    """Calculates term-wise statistics for the given DataFrame"""
    mme = list(df.filter(regex='mme-').columns)
    models = list(df.columns.drop(['variable', 'scenario', 'term', 'years'] + mme))
    
    df1 = df.set_index(['variable', 'scenario', 'term', 'years'])
    df_mme = df1.filter(regex='mme-').round(5).copy()
    
    df1['mme-mean'] = df1[models].mean(axis=1)
    df1['mme-median'] = df1[models].median(axis=1)
    df1['mme-pct25'] = df1[models].quantile(0.25, axis=1)
    df1['mme-pct75'] = df1[models].quantile(0.75, axis=1)
    df1['mme-pct05'] = df1[models].quantile(0.05, axis=1)
    df1['mme-pct95'] = df1[models].quantile(0.95, axis=1)
    
    df_cal = df1.filter(regex='mme-')[df_mme.columns].round(5)
    
    err = df_mme.compare(df_cal, result_names=('mme', 'recalc')).dropna()
    if len(err) > 0:
        display(err)
        raise ValueError('Calculation mismatch')

def aggregate_data(df: pd.DataFrame):
    """Aggregates data for real and days variables"""
    mme = list(df.filter(regex='mme-').columns)
    cols = ['variable', 'scenario', 'term']

    # Calculate term-wise statistics
    # For variables ending with '_days', use median
    # For variables ending with '_real', use mean
    term_mme = pd.concat([df[df.variable.str.endswith('_days')].groupby(cols)[mme].median(),
                          df[df.variable.str.endswith('_real')].groupby(cols)[mme].mean()
                         ]).reset_index().sort_values(cols, ascending=[1, 1, 0],
                                                      ignore_index=True)

    round_up_half = lambda x: np.ceil(x) if x % 1 == 0.5 else round(x)
    agg_real = (term_mme[term_mme.variable.str.endswith('_real')]
                .groupby(['variable', 'term'])
                .agg({'mme-mean': ['min', 'max']})
                .sort_values(['variable', 'term'], ascending=[1, 0]))
    agg_real.columns = ['min', 'max']
    agg_real['rounded_min'], agg_real['rounded_max'] = agg_real['min'].round(1), agg_real['max'].round(1)
    agg_real['difference'] = (agg_real['rounded_max'] - agg_real['rounded_min']).round(1)
    
    agg_days = (term_mme[term_mme.variable.str.endswith('_days')]
                .groupby(['variable', 'term'])
                .agg({'mme-median': ['min', 'max']})
                .sort_values(by=['variable', 'term'], ascending=[1, 0]))
    agg_days.columns = ['min', 'max']
    agg_days['rounded_min'] = agg_days['min'].apply(round_up_half).astype(int)
    agg_days['rounded_max'] = agg_days['max'].apply(round_up_half).astype(int)
    agg_days['difference'] = (agg_days['rounded_max'] - agg_days['rounded_min']).astype(int)
    
    return pd.concat([agg_real, agg_days], axis=0)

def get_results(path:str, center: str):
    """Writes aggregated data for a given center to an Excel file"""    
    files = sorted(get_files(path, center))
    df_list = [preprocess(f, only_future) for f in files]
    
    # if not check_df_consistency(df_list):
    #     raise ValueError('DataFrames are inconsistent')
    
    df = pd.concat(df_list).reset_index(drop=True)
    
    # Check the number of years per time period
    years_per_term = df.groupby(['variable', 'scenario', 'term']).size().unique()
    if len(years_per_term) != 1 or years_per_term[0] != 30:
        raise ValueError(f'# of years per time period is incorrect: {years_per_term}')
    
    calculate_statistics(df)
    print(f'{len(files)} {center} files')
    # print(files[:5], '\n')
    return df, aggregate_data(df)

In [5]:
df, results = get_results(path, center)

69 LARC files


# Quality Check

In [6]:
df.groupby('variable').max()

,scenario,term,years,access-cm2,access-esm1-5,cnrm-cm6-1,cnrm-esm2-1,ec-earth3,fgoals-g3,gfdl-esm4,...,noresm2-mm,mme-mean,mme-median,mme-pct25,mme-pct75,bcc-csm2-mr,cesm2,cmcc-esm2,mme-pct05,mme-pct95
variable,,,,,,,,,,,,,,,,,,,,,
Coldest_Tmin_real,ssp370,short,2099.0,-0.931519,-3.823822,-3.291687,-4.559418,-3.377991,-3.947113,-2.566589,...,-3.331390,-6.561266,-6.352448,-7.718903,-4.259308,-2.503448,NaN,-2.908722,-10.060699,-1.363556
FWI_N15_500km_days,ssp370,short,2099.0,18.469492,30.422566,37.494746,43.423570,44.723364,3.097849,28.122191,...,33.355984,11.154803,5.411148,1.806614,14.658094,NaN,NaN,NaN,NaN,NaN
FWI_N15_Center_days,ssp370,short,2099.0,11.000000,32.000000,58.000000,65.000000,28.000000,2.000000,16.000000,...,19.000000,7.684211,2.000000,0.000000,9.000000,NaN,NaN,NaN,NaN,NaN
FWI_N30_500km_days,ssp370,short,2099.0,2.659874,3.651951,2.682407,4.776031,10.315885,0.140476,2.190971,...,2.725350,1.717636,0.027278,0.000000,0.517364,NaN,NaN,NaN,NaN,NaN
FWI_N30_Center_days,ssp370,short,2099.0,1.000000,2.000000,4.000000,6.000000,1.000000,0.000000,1.000000,...,0.000000,0.315789,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
FWI_N45_500km_days,ssp370,short,2099.0,0.149651,0.119972,0.049641,0.236870,1.095489,0.000000,0.089583,...,0.168659,0.314002,0.000000,0.000000,0.014115,NaN,NaN,NaN,NaN,NaN
FWI_N45_Center_days,ssp370,short,2099.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.052632,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
Hottest_Tmax_real,ssp370,short,2099.0,47.414276,45.155914,46.238922,47.965546,48.440369,44.046844,44.450012,...,47.794220,42.553859,42.776978,41.717285,43.820068,47.664703,NaN,45.756561,39.927338,47.414276
Max_DTR_real,ssp370,short,2099.0,34.059143,29.701355,37.473969,33.203094,32.599701,38.783783,35.950317,...,32.301270,26.489332,27.029785,24.990265,29.414246,39.230255,NaN,31.971222,23.054901,37.861099


In [7]:
df.groupby('variable').min()

,scenario,term,years,access-cm2,access-esm1-5,cnrm-cm6-1,cnrm-esm2-1,ec-earth3,fgoals-g3,gfdl-esm4,...,noresm2-mm,mme-mean,mme-median,mme-pct25,mme-pct75,bcc-csm2-mr,cesm2,cmcc-esm2,mme-pct05,mme-pct95
variable,,,,,,,,,,,,,,,,,,,,,
Coldest_Tmin_real,ssp126,long,2020.0,-20.428391,-20.410950,-24.557144,-22.943710,-22.828995,-18.954742,-19.032974,...,-21.732437,-13.303314,-13.215698,-15.710175,-10.944702,-30.617416,NaN,-19.131744,-23.594238,-9.756409
FWI_N15_500km_days,ssp126,long,2020.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.299920,0.047646,0.006254,0.208416,NaN,NaN,NaN,NaN,NaN
FWI_N15_Center_days,ssp126,long,2020.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.105263,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
FWI_N30_500km_days,ssp126,long,2020.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
FWI_N30_Center_days,ssp126,long,2020.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
FWI_N45_500km_days,ssp126,long,2020.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
FWI_N45_Center_days,ssp126,long,2020.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
Hottest_Tmax_real,ssp126,long,2020.0,36.378784,34.746796,34.750946,35.739655,35.460693,34.615631,35.314667,...,36.164886,37.705517,37.221222,35.866425,38.633484,36.579468,NaN,35.086365,33.838257,39.813721
Max_DTR_real,ssp126,long,2020.0,19.484528,20.472870,20.811005,19.885498,19.565277,18.565277,20.344452,...,20.936829,23.059900,22.423767,21.014496,23.827087,20.711517,NaN,19.935699,19.194702,25.837036


## Calculate Change Per (variable, scenario)
- short - mid
- short - long
- mid - long

In [8]:
# query = """
# SELECT
#     a.variable,
#     a.scenario,
#     CASE
#         WHEN a.term = 'short' AND b.term = 'mid' THEN 'short-mid'
#         WHEN a.term = 'short' AND b.term = 'long' THEN 'short-long'
#         WHEN a.term = 'mid' AND b.term = 'long' THEN 'mid-long'
#     END AS term_diff,
#     b.'mme-mean' - a.'mme-mean' AS 'mme-mean',
#     b.'mme-median' - a.'mme-median' AS 'mme-median',
#     b.'mme-pct25' - a.'mme-pct25' AS 'mme-pct25',
#     b.'mme-pct75' - a.'mme-pct75' AS 'mme-pct75'
# FROM term_mme a
# JOIN term_mme b
#     ON a.variable = b.variable
#     AND a.scenario = b.scenario
#     AND (
#         (a.term = 'short' AND b.term = 'mid') OR
#         (a.term = 'short' AND b.term = 'long') OR
#         (a.term = 'mid' AND b.term = 'long')
#     )
# ORDER BY 1, 2, 3 DESC
# """

# change = psql.sqldf(query, locals())

# display(change.head(2))

# Results

In [9]:
pd.set_option('display.max_rows', None)

display(results)

min          max  rounded_min  \
variable                 term                                           
Coldest_Tmin_real        short   -11.475012   -11.276643        -11.5   
                         mid     -10.754145   -10.200959        -10.8   
                         long    -10.463190    -7.920878        -10.5   
Hottest_Tmax_real        short    38.696575    39.078562         38.7   
                         mid      39.179674    39.696653         39.2   
                         long     39.126319    41.202436         39.1   
Max_DTR_real             short    24.576354    24.833572         24.6   
                         mid      24.486292    24.680787         24.5   
                         long     24.232885    24.449889         24.2   
annmean_tmax_real        short    22.712597    22.799994         22.7   
                         mid      23.029034    23.470423         23.0   
                         long     23.120826    24.822218         23.1   
annmean_tmin_real        short    11.535365    11.632732         11.5   
                         mid      11.850220    12.386017         11.9   
                         long     11.934369    13.833739         11.9   
annsum_prec_real         short  1246.330021  1261.840604       1246.3   
                         mid    1269.593120  1273.584706       1269.6   
                         long   1284.426721  1323.865048       1284.4   
FWI_N15_500km_days       short     0.208079     0.450783          0.0   
                         mid       0.221935     1.242755          0.0   
                         long      0.224064     3.135400          0.0   
FWI_N15_Center_days      short     0.000000     0.000000          0.0   
                         mid       0.000000     0.000000          0.0   
                         long      0.000000     0.000000          0.0   
FWI_N30_500km_days       short     0.000000     0.000000          0.0   
                         mid       0.000000     0.000000          0.0   
                         long      0.000000     0.002664          0.0   
FWI_N30_Center_days      short     0.000000     0.000000          0.0   
                         mid       0.000000     0.000000          0.0   
                         long      0.000000     0.000000          0.0   
FWI_N45_500km_days       short     0.000000     0.000000          0.0   
                         mid       0.000000     0.000000          0.0   
                         long      0.000000     0.000000          0.0   
FWI_N45_Center_days      short     0.000000     0.000000          0.0   
                         mid       0.000000     0.000000          0.0   
                         long      0.000000     0.000000          0.0   
prec_days_90th_days      short    24.750000    25.500000         25.0   
                         mid      26.000000    26.000000         26.0   
                         long     26.000000    27.500000         26.0   
prec_days_95th_days      short    13.000000    13.000000         13.0   
                         mid      13.250000    13.500000         13.0   
                         long     13.000000    14.750000         13.0   
prec_days_99th_days      short     3.000000     3.000000          3.0   
                         mid       3.000000     3.000000          3.0   
                         long      3.000000     4.000000          3.0   
prec_days_dry_days       short   119.250000   122.250000        119.0   
                         mid     117.250000   121.250000        117.0   
                         long    119.750000   122.500000        120.0   
prec_days_oneinch_days   short     6.000000     6.000000          6.0   
                         mid       6.000000     7.000000          6.0   
                         long      6.500000     7.750000          7.0   
tmax_days_35C_days       short    14.000000    15.000000         14.0   
                         mid      16.000000    21.500000         16.0   
                         

# All

In [10]:
with pd.ExcelWriter('data/center_casi_projections.xlsx') as writer:
    for center in centers:
        df, results = get_results(path, center)
        results.to_excel(writer, sheet_name=center)

69 AFRC files
69 AMES files
69 GISS files
69 GRC files
69 GSFC files
69 JPL files
69 JSC files
69 KSC files
69 LARC files
69 MAF files
69 MSFC files
69 SSC files
69 WFF files
69 WSTF files
